# Algoritmos de Machine Learning
Este trabalho consiste na utilização de algoritmos de machine learning com a finalidade de criar modelos (preditivos, escritivos ou híbridos) que possibilitem extrair padrões ou conhecimento do dataset utilizado nos trabalhos anteriores.

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly 
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from keras.models import model_from_json
#plotly.offline.init_notbook_mode()

In [2]:
df = pd.read_csv('csvfile_preprocessado.csv', usecols = ['ANO_BO', 
                                                        'NUM_BO', 
                                                        'DATAOCORRENCIA',
                                                        'HORAOCORRENCIA',
                                                        'HORARIOOCORRENCIA',
                                                        'PERIDOOCORRENCIA',
                                                        'BAIRRO', 
                                                        'CIDADE',
                                                        'LOGRADOURO', 
                                                        'DESCRICAOLOCAL', 
                                                        'DESCR_TIPO_VEICULO', 
                                                        'DELEGACIA_NOME',
                                                        'FLAGRANTE',
                                                        ], 
    encoding='utf-8', 
)

In [3]:
# Transformar a coluna DATAOCORRENCIA em uma coluna do tipo Datetime
df['DATAOCORRENCIA'] = pd.to_datetime(df['DATAOCORRENCIA'], format = '%Y-%m-%d') #%d-%m-%Y')

In [20]:
df['HORAOCORRENCIA'] = pd.to_datetime(df['HORAOCORRENCIA'], format = '%H:%M:%S').dt.time

In [5]:
df['MES'] = df['DATAOCORRENCIA'].apply(lambda x: int(str(x)[5:7]))

In [22]:
#df.head()
#df.tail()
#df.info()

In [23]:
entrada = df['HORAOCORRENCIA'].to_numpy()
saidas = df['PERIDOOCORRENCIA']

valoresSaida = np.empty((4276, 1), dtype=int) #vetor

for i in range(4276):
   valoresSaida[i]= saidas[i]-1

entradasTrain, entradasTest, valoresSaidaTrain, valoresSaidaTest = train_test_split(entradas, valoresSaida, test_size=0.3)

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
print("Quantidade de Dados de Treino:" ,len(entradasTrain))
print("Quantidade de Dados de Teste:" ,len(entradasTest))

print("Quantidade de Dados de Treino e dos atributos:" ,entradasTrain.shape)
print("Quantidade de Dados de Teste e dos atributos" ,entradasTest.shape)

print("Quantidade de Saidas de Treino e coluna:" ,valoresSaidaTrain.shape)
print("Quantidade de Saidas de Teste e coluna:" ,valoresSaidaTest.shape)

print("min: ", valoresSaidaTrain.min())
print("max: ", valoresSaidaTrain.max())

In [ ]:
#aqui eu calculo o meu medelo
modelo = keras.Sequential([keras.layers.Dropout(0.2),
                           keras.layers.Dense(130, activation=tensorflow.nn.relu), # a deeplearn comecei com valores
                           keras.layers.Dense(70, activation=tensorflow.nn.relu),  # para a ver a minha acuracia aumentar
                           keras.layers.Dense(40, activation=tensorflow.nn.relu),  # usandoo função relu
                           keras.layers.Dense(13, activation=tensorflow.nn.relu),
                           keras.layers.Dense(3, activation=tensorflow.nn.softmax) # saida
                           ]) 

modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# aqui eu faço o treinamento da minha rede, passando os valores de treino e de saída, epocas e validação
hist=modelo.fit(entradasTrain, valoresSaidaTrain, epochs=70, validation_split=0.3)

In [ ]:
# aqui peço para plotar o meu grafico
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Acurácia por épocas')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend(['Treino', 'Validação'])

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss por épocas')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend(['Treino', 'Validação'])

In [ ]:
# aqui estou salvando o meu modelo para poder reurar em outros datasets
model_json = modelo.to_json()
with open("model.json", "w") as json_file:
   json_file.write(model_json)

modelo.save_weights("model.h5")
print("Modelo Salvo")

json_file = open("model.json", "r")
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Modelo Carregado")

loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

perda_teste, acuracia_teste = loaded_model.evaluate(entradasTest, valoresSaidaTest)
print("Perda do teste: ", perda_teste)
print("Acurácia do teste: ", acuracia_teste)
testes=loaded_model.predict(entradasTest)

x=0
for i in testes:
   print(np.argmax(i), valoresSaidaTest[x])
   x=x+1